<a href="https://colab.research.google.com/github/Piontk/Oracle-Database-Inverted-File-Flat-IVF-indexes-/blob/main/IVF_parte2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain langchain-community langchain-openai langchain-oracledb pypdf oracledb python-dotenv

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
CONNECT_STRING = os.getenv("CONNECT_STRING")
WALLET_PASSWORD = os.getenv("WALLET_PASSWORD")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [ ]:
pergunta = "How to grant a role locally?"

In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)
vector = embeddings.embed_query(pergunta)

In [ ]:
print(vector)

[0.0009422260336577892, -0.005045970901846886, -0.0005247567314654589, -0.032615382224321365, -0.030345043167471886, 0.022011231631040573, -0.01940864510834217, 0.01502024196088314, -0.012798354029655457, -0.025416741147637367, 0.031784769147634506, -0.033058375120162964, 0.0036927645560353994, 0.003571633715182543, -0.014964868314564228, 0.0010988310677930713, 0.0207514688372612, -0.015297112986445427, 0.011960819363594055, -0.022246571257710457, -0.01543554849922657, -0.003575094509869814, 0.018882591277360916, -0.0088737104088068, -0.006042705848813057, 0.014826432801783085, 0.011794697493314743, -0.024516912177205086, 0.02311871387064457, -0.007392451167106628, 0.032836880534887314, -0.005758913233876228, -0.005118649452924728, 0.0004016790189780295, -0.010403421707451344, 0.040589261800050735, 0.008949849754571915, -0.006769491825252771, -0.007717774249613285, 0.020350005477666855, 0.014286534860730171, -0.0071086585521698, 0.004301880486309528, -0.04438239336013794, 0.01846728473

In [ ]:
import oracledb
import array

# setar a fetch_lobs para False para retornar o CLOB como texto
oracledb.defaults.fetch_lobs = False

# conexão
conn = oracledb.connect(user=DB_USER,
                              password=DB_PASSWORD,
                              dsn=CONNECT_STRING,
                              config_dir="/content/wallet",
                              wallet_location="/content/wallet",
                              wallet_password=WALLET_PASSWORD )

# Transformar o vetor em array float32 para a bind variable
vector_arr = array.array("f", vector)

cursor = conn.cursor()

sql = f"""
SELECT /*teste_IVF*/
       text,
       JSON_VALUE(metadata, '$.source') AS source,
       vector_distance(embedding, TO_VECTOR(:vec), EUCLIDEAN) as distance
FROM "ORACLE_DOCS"
ORDER BY distance
FETCH APPROX FIRST 5 ROWS ONLY
"""

# execução do SQL repassando o vetor para a bind
cursor.execute(sql, {"vec": vector_arr})

rows = cursor.fetchall()
for row in rows:
    print(f'Texto:     {row[0]}')
    print(f'Fonte:     {row[1]}')
    print(f'Distancia: {row[2]}')
    print('\n\n')

Texto:     The Roles page appears.
3. Click Create.
If prompted, enter your login information. Afterward, the Create Role page appears.
4. Select the options that create a local role and grant this role privileges.
Ensure that you do not preface the role name with C## or c##.
5. Click OK.
The local role is created in the current PDB.
Related Topics
• Logging into a Multitenant Environment in Enterprise Manager
You can log in to a CDB or a PDB, and switch from a PDB to a different PDB or to the root.
• Granting or Revoking Privileges to Access a PDB
You can grant and revoke privileges for PDB access.
11.4.6 Editing a Local Role in Enterprise Manager
You can edit a local role in the PDB in which the local role resides.
1. From the Enterprise Manager database home page, log in to the PDB as a user who has
the local CREATE ROLE privilege.
2. From the Administration menu, select Security, then Roles.
If prompted, enter your login information. Afterward, the Roles page appears, showing only
